In [ ]:
import sys
import pytorch_lightning as pl
sys.path.append('../input/timmmaster')
import os
import json
import glob
import random
import collections
import albumentations as A
import timm
import cuml ,pickle
from cuml.svm import SVR
from cuml.linear_model import LogisticRegression, ElasticNet
from cuml.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch
package_path = "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/"
import sys 
sys.path.append(package_path)
import glob
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold

import time
import os
import torch
from torch import nn
from torch.cuda import amp
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import efficientnet_pytorch
import torch.optim.lr_scheduler as lr_scheduler

from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train_df
# train_df_extr=pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
# train_df_extr['Pawpularity']=train_df_extr['AdoptionSpeed'].apply(lambda x: float(abs(x*25-5)))
# train_df_extr

In [ ]:
submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
submission.head()
submission.Pawpularity.values/100


# Set Seed Config

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(42)

In [ ]:
df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")


# Transformations

In [ ]:
import albumentations

transform = albumentations.Compose(
     [   
       
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],)

# Dataset 

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(self,ids,targets,train=False,valid=False):
        self.id=ids
        self.targets=targets
        self.train=train
        self.valid=valid
        self.alpha=0.2
        
    def __len__(self):
        return len(self.id)
    def __getitem__(self,index):
        if self.train or self.valid:
            img_path='../input/petfinder-pawpularity-score/train/'+str(self.id[index])+'.jpg'
        else:
            img_path='../input/petfinder-pawpularity-score/test/'+str(self.id[index])+'.jpg'
            
        
        if self.train or self.valid:
            labels=torch.tensor(self.targets[index],dtype=torch.float)
        img=cv2.imread(img_path)
        
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,(224,224))
        
        if self.train:
            img=transform(image=img)['image']
#             if index%2!=0:
#                 lam=np.random.beta(self.alpha,self.alpha)
#                 mixup_index=np.random.randint(0,index)
#                 labels2=torch.tensor(self.targets[mixup_index],dtype=torch.float)
#                 img_path='../input/petfinder-pawpularity-score/train/'+str(self.id[mixup_index])+'.jpg'
#                 img2=cv2.imread(img_path)
#                 img2=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#                 img2=cv2.resize(img,(224,224))
#                 img=lam*img+(1-lam)*img2
#                 labels=lam*labels+(1-lam)*labels2
        else:
            img=valid_aug(image=img)['image']
      
        img=np.transpose(img,(2,0,1)).astype(np.float32)
        if self.train or self.valid:
            return {
            'X':torch.tensor(img,dtype=torch.float),   
            'y':labels
          }
        return{'X':torch.tensor(img,dtype=torch.float)} 

# Practice Dataset

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()
        
    def forward(self,x,y):
        criterion=nn.MSELoss()
        loss=torch.sqrt(criterion(x,y))
        return loss
    

# Model

In [ ]:
def train(model,criterion,metric,scheduler,optimizer,epochs,train_loader,valid_loader):
    
    scaler=amp.GradScaler()
    for epoch in range(epochs):
        total_loss=[]
        total_rmse=[]
        total_val_rmse=[]
        model.train()
        loop=tqdm(enumerate(train_loader),total=len(train_loader))
        for i,batch in loop:
            img=batch['X'].to('cuda')
            y=batch['y'].to('cuda')
            y=y.view(-1,1)
            optimizer.zero_grad()
            with amp.autocast():
                output=model(img)
                loss=criterion(output,y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
#             output=model(img)
#             loss=criterion(output,y)
#             loss.backward()
#             optimizer.step()
            rmse=metric(torch.sigmoid(output)*100,y*100)
            
            total_loss.append(loss.item())
            total_rmse.append(rmse.detach().cpu().numpy())
            train_rmse=sum(total_rmse)/len(total_rmse)
            loop.set_postfix(Train_Rmse_loss=train_rmse)
        model.eval()
        with torch.no_grad():
            loop=tqdm(enumerate(valid_loader),total=len(valid_loader)) 
            for i,batch in loop:
                img=batch['X'].to('cuda')
                y=batch['y'].to('cuda')
                y=y.view(-1,1)
                output=model(img)
               
                rmse=metric(torch.sigmoid(output)*100,y*100)
                total_val_rmse.append(rmse.detach().cpu().numpy())
                val_rmse=sum(total_val_rmse)/len(total_val_rmse)
                loop.set_postfix(Valid_Rmse_loss=val_rmse)
        
       
        #scheduler.step()
        print(len(total_rmse))
        train_rmse=sum(total_rmse)/len(total_rmse)
        val_rmse=sum(total_val_rmse)/len(total_val_rmse)
        print(f"Epoch-> {epoch}  Train Rmse->>{train_rmse}   Val_rmse-->>{val_rmse} ")
        
   

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('swin_large_patch4_window7_224_in22k', pretrained=False, in_chans=3)
        #self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128, 1)
        #self.dense2 = nn.Linear(64, 1)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        #self.model.fc = nn.Linear(self.model.fc.in_features, 128)
        
    def forward(self,x):
        output=self.model(x)
        #print(output.shape)
        output=self.dropout(output)
        #print(output.shape)
        #output=torch.cat([output,dense_features],dim=1)
        output=self.dense1(output)
        #print(output.shape)
        #output=self.dense2(output)
       # output=torch.sigmoid(output)       
        return output
    


In [ ]:
def get_dataloaders(train_idx,val_idx):


    valid_data_retriever=DataRetriever(df.Id.values[val_idx],
                               df.Pawpularity.values[val_idx]/100,
                                           train=False,valid=True)
    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=32,
        shuffle=False,
        num_workers=8,
        )
    
    train_data_retriever=DataRetriever(df.Id.values[train_idx],
                               df.Pawpularity.values[train_idx]/100,
                                           train=True,valid=False)
    
    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=32,
        shuffle=True,
        num_workers=8,
        )
    return train_loader,valid_loader
    

In [ ]:
def get_settings():
    device='cuda'
   # criterion=nn.BCELoss()
    criterion =nn.BCEWithLogitsLoss()
    #criterion=LabelSmoothingCrossEntropy()
    #criterion = FocalLoss()
    metric=RMSELoss()
    model=PawpularityModel()
    model.to(device)
    optimizer =torch.optim.AdamW(model.parameters(), lr=2e-6,
                                  amsgrad=False)
#     scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
#             optimizer,
#             T_0=5,
#             eta_min=1e-7,
#             last_epoch=-1
#         )
    scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-4, last_epoch=-1
        )
    #scheduler=torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-5)
    return criterion,metric,model,optimizer,scheduler

# Training

In [ ]:
# df['fold']=-1
# num_bins=int(np.floor(1+np.log2(len(df))))
# df.loc[:,'bins']=pd.cut(df['Pawpularity'],bins=num_bins,labels=False)
# kf=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
# epochs=2
# for fold,(train_idx,val_idx) in enumerate(kf.split(df.index,df.Pawpularity.values)):
#     print(f"Fold-->>{fold}")
#     criterion,metric,model,optimizer,scheduler= get_settings()
#     train_loader,valid_loader= get_dataloaders(train_idx,val_idx)
#     train(model,criterion,metric,scheduler,optimizer,epochs,train_loader,valid_loader)
#     torch.save(model.state_dict(),f"model_fold={fold}.pth")
#     print('saved')

# Submission

In [ ]:
submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
model=PawpularityModel()
test=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
model.to('cuda')
device='cuda'
test_data_retriever = DataRetriever(
    submission.Id.values,None,train=False,
    valid=False
)

test_loader = torch_data.DataLoader(
    test_data_retriever,
    batch_size=64,
    shuffle=False,
    num_workers=8,
)

In [ ]:
ids = []
paths=[]
pred=[]
for path in os.listdir('../input/pet-paw-data'):
    paths.append('../input/pet-paw-data/'+path)

for path in paths:
    y_pred = []
    model.load_state_dict(torch.load(path))
    for e, batch in enumerate(test_loader):
        print(f"{e}/{len(test_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = np.zeros((batch["X"].shape[0], ))

            tmp_pred=model(batch['X'].to(device)).cpu().numpy().squeeze()
            #print(tmp_pred)
            tmp_pred=1/(1+np.exp(-tmp_pred))
           # print(tmp_pred)
            
            y_pred.extend(tmp_pred)
    y_pred=[x*100 for x in y_pred]
    pred.append(y_pred)
    
pred=np.mean(np.column_stack(pred), axis=1)    

In [ ]:
submission = pd.DataFrame({"Id": submission.Id.values, "Pawpularity": pred})
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head()